In [ ]:
#ALL INSTALLTIONS
!pip install transformers

In [ ]:
#Custom Data set and Data loader has  been adapted and inspired from 
#Michael Sugimura,Github Repository:https://github.com/sugi-chan/custom_bert_pipeline
#BERT based fine tuning adapted and inspired from:Chris McCormick and Nick Ryan. (2019, July 22). BERT Fine-Tuning #Tutorial with PyTorch. Retrieved from http://www.mccormickml.com
#for all references refer README.md

In [4]:

import tensorflow as tf
import torch

import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

from sklearn.svm import SVC
from sklearn import preprocessing

from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig


Using TensorFlow backend.


In [0]:
gpuname=""
device=""
y=""
preprocessedTweets=""
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]


In [0]:

def initGpus1():
  gpuname=tf.test.gpu_device_name()
  if gpuname=='/device:GPU:0':
    print('Found GPU at :{}'.format(gpuname))
  else:
    gpuname=""
  if torch.cuda.is_available():
    device=torch.device("cuda")
    n_gpu=torch.cuda.device_count()
    print("The device name is %s"%torch.cuda.get_device_name(0))
  else:
    print("No GPU available using only CPU instead")
    device=torch.device("cpu")
  return gpuname,device



In [ ]:

from google.colab import drive
drive.mount('/content/drive')


In [0]:
!unzip -P yourpassword -qq '/content/drive/My Drive/GreekData/PredictFile.zip'

In [0]:
##Use blob
#GET THE DATA FROM THE PANDAS FRAME
def readData1():
  headers=['tweet','subtask_a']
  greekdata = pd.read_csv("PredictFile.csv", delimiter=',',names=headers)
  
  data=greekdata[1:]
  dfnumpy=data.to_numpy();
  X=dfnumpy[:, 0].reshape(-1, 1)
  y=dfnumpy[:, 1].reshape(-1, 1)
  arrt=X[:,0]
  #print(X)
  #print(y)
  preprocessedTweets=arrt
  return preprocessedTweets,y

In [0]:
#preprocessedTweets,y=readData1()

In [11]:
'''
##Use blob
#GET THE DATA FROM THE PANDAS FRAME
def readData1():
  headers=['tweet','subtask_a']
  greekdata = pd.read_csv("PredictFile.csv", delimiter='\t',names=headers)
  data=greekdata[1:]
  #print(greekdata)
  dfnumpy=data.to_numpy();
  X=dfnumpy[:, 0].reshape(-1, 1)


  #REMOVE THE BELOW LINE
  greekdata1 = pd.read_csv("PredictFile.csv", delimiter=',',names=headers)

  data=greekdata[1:]
  dfnumpy=data.to_numpy();
  X=dfnumpy[:, 0].reshape(-1, 1)
  #print(X)
  
  data=greekdata1[1:]
  dfnumpy=data.to_numpy();
  y=dfnumpy[:, 1].reshape(-1, 1)
  
  arrt=X[:,0]
  #print(y)
  preprocessedTweets=arrt

  return preprocessedTweets,y
  '''

'\n##Use blob\n#GET THE DATA FROM THE PANDAS FRAME\ndef readData1():\n  headers=[\'tweet\',\'subtask_a\']\n  greekdata = pd.read_csv("PredictFile.csv", delimiter=\'\t\',names=headers)\n  data=greekdata[1:]\n  #print(greekdata)\n  dfnumpy=data.to_numpy();\n  X=dfnumpy[:, 0].reshape(-1, 1)\n\n\n  #REMOVE THE BELOW LINE\n  greekdata1 = pd.read_csv("PredictFile.csv", delimiter=\',\',names=headers)\n\n  data=greekdata[1:]\n  dfnumpy=data.to_numpy();\n  X=dfnumpy[:, 0].reshape(-1, 1)\n  #print(X)\n  \n  data=greekdata1[1:]\n  dfnumpy=data.to_numpy();\n  y=dfnumpy[:, 1].reshape(-1, 1)\n  \n  arrt=X[:,0]\n  #print(y)\n  preprocessedTweets=arrt\n\n  return preprocessedTweets,y\n  '

In [0]:
def giveIds(sentence):
  tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      ids_of_sentence.append(tokenized_sentence_id)
  print(maxlength)
  ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length
  attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] 
  return ids_of_sentence_words,attention_masks

In [13]:
xpredict,xpredictmask=giveIds(preprocessedTweets)


0


In [14]:
ids_of_sentence_words

[]

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  #print("predictionArgmax",predictionArgmax)
  #print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [0]:
def predictingData(pTweets,ypred):
  ids_of_sentence=[]
  predictedLabels,trueLabels=[],[]
  
  xpredict,xpredictmask=giveIds(pTweets)
  le = preprocessing.LabelEncoder()
  ypredict=le.fit_transform(ypred.flatten())

  x_predict_pytorch=torch.tensor(xpredict)
  y_predict_pytorch=torch.tensor(ypredict)
  x_predict_mask_pytorch=torch.tensor(xpredictmask)

  bsize = 32
  predictdata=TensorDataset(x_predict_pytorch,x_predict_mask_pytorch,y_predict_pytorch)
  predictsampler=RandomSampler(predictdata)
  predictdataloader=DataLoader(predictdata,sampler=predictsampler,batch_size=bsize)
  
  model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
  model.cuda()
  params=list(model.named_parameters())
  
  eval_f1=0
  eval_acc=0
  nb_eval_steps=0
  checkpoint = torch.load('/content/drive/My Drive/GreekData/bertgreek.pth.tar')
  model.load_state_dict(checkpoint['state_dict'])
  model.eval()

  for batch in predictdataloader: 
      batch = tuple(t.to(device) for t in batch)        
      inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
      with torch.no_grad():       
          outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = (inputs["labels"]).to('cpu').numpy()
      predictedLabels.append(logits)
      trueLabels.append(label_ids)
      tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
      eval_f1 = eval_f1+tmpf1score
      eval_acc=eval_acc+tmpaccscore
      nb_eval_steps += 1
      
  print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
  print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
  return predictedLabels,trueLabels

In [0]:
#device

In [63]:
def main():
  initGpus1()  
  preprocessedTweets,y=readData1()
  predictedLabels,trueLabels=predictingData(preprocessedTweets,y)

if __name__ == "__main__":
    main()

Found GPU at :/device:GPU:0
The device name is Tesla P100-PCIE-16GB
166
  F1 score: 0.839
  Accuracy score: 0.883
